In [ ]:
import os
import time
import pandas as pd
from dotenv import load_dotenv
from langchain.prompts import PromptTemplate
from langchain.chat_models.gigachat import GigaChat
from langchain_core.output_parsers import StrOutputParser

In [ ]:
load_dotenv()

GIGACHAT_AUTH = os.getenv('GIGACHAT_AUTH')

In [ ]:
# Инициализация llm 
llm = GigaChat(model = "GigaChat-Pro", # "GigaChat-Pro" / "GigaChat-Plus" (32k)
               temperature = 0.2,
               credentials = GIGACHAT_AUTH, 
               scope = 'GIGACHAT_API_CORP',
               verify_ssl_certs = False)

In [ ]:
# Загружаем данные
data = pd.read_csv('data/balanced.csv')

In [ ]:
# Парсер
parser = StrOutputParser()

# Промпт для дополненной генерации
template = """
Вы эксперт в области обработки информации.

Ваша задача: На основе предоставленного отзыва определите и структурируйте информацию о заведении. 
             Разделите текст на две части: сильные стороны и слабые стороны.

        Инструкция:

        1. Прочтите текст отзыва.
        2. Выделите конкретные аспекты заведения, упомянутые в отзыве (например, обслуживание, качество продукта, атмосфера).
        3. Отнесите каждый аспект к одной из двух категорий:

                - Сильные стороны: аспекты, которые были положительно оценены в отзыве.
                - Слабые стороны: аспекты, которые вызвали негативные эмоции или упоминания о недостатках.

Если отзыв содержит общие фразы без конкретики (например, "всё хорошо" или "не понравилось" или "в целом понравилось"), 
укажите это как "нет конкретики" в соответствующей категории.
Используйте только те аспекты, которые явно упомянуты в отзыве. Не делайте предположений, если информация отсутствует. 

                Отзыв: {post}

Ваш ответ должен быть структурирован строго в виде списка:

        Сильные стороны:
                1. ...
                2. ...

        Слабые стороны:
                1. ...
                2. ...

Важно: Только сильные и слабые стороны, без упоминания отзыва.
"""

prompt = PromptTemplate(template=template, input_variables=['post'])

# Цепь генерации 
chain = prompt | llm | parser

In [ ]:
# Проверяем вывод модели 
sample = data['clean_text'].loc[12000]
result = chain.invoke({"post":sample})
print(result)

**Сильные стороны:**
1. Чистота в номерах.
2. Уровень уборки номеров.
3. Прекрасное расположение (до моря 5 минут).
4. Хорошая территория.
5. В целом всё понравилось.

**Слабые стороны:**
1. Номерной фонд немного ушатан.
2. Качество еды на ужине оценивается на троечку.
3. Еда на ужин однообразная, ощущается, что подают остатки с завтрака и обеда.


In [ ]:
# Проверка наличия столбца 'synthetic' 
if 'synthetic' not in data.columns:
    data['synthetic'] = ""

# Максимальное количество попыток и задержка между ними
MAX_RETRIES = 5
RETRY_DELAY = 5

# Обработка каждой строки данных
for index, row in data.iterrows():
    # Пропуск строк, которые уже обработаны
    if pd.notna(row['synthetic']) and row['synthetic'].strip() != "":
        print(f"Row {index} already processed. Skipping.")
        continue

    attempt = 0
    while attempt < MAX_RETRIES:
        try:
            # Текст для обработки из столбца 'clean_text'
            sample = row['clean_text']
            print(f"Processing row {index} (Attempt {attempt + 1})...")
            
            # Вызов llm для обработки текста
            result = chain.invoke({"post": sample})
            
            # Сохранение результата
            data.at[index, 'synthetic'] = result
            
            # Сохранение данных
            data.to_csv('data/synthetic.csv', index=False)
            print(f"Row {index} processed successfully.")
            break 
        
        except Exception as e:
            attempt += 1
            print(f"Error processing row {index} on attempt {attempt}: {e}")
            
            if attempt < MAX_RETRIES:
                # Повторная попытка после задержки
                print(f"Retrying row {index} after {RETRY_DELAY} seconds...")
                time.sleep(RETRY_DELAY)
            else:
                # Если все попытки исчерпаны, помечаем строку как "ERROR"
                print(f"Failed to process row {index} after {MAX_RETRIES} attempts. Skipping.")
                data.at[index, 'synthetic'] = "ERROR"
                data.to_csv('data/synthetic.csv', index=False)

# Завершаем
print("Saved to 'data/synthetic.csv'.")